In [192]:
import tweepy
import numpy as np
import time

In [157]:
DATA_PATH = "./id_data/train.data.txt"
JSON_PATH = "./full_data/full_train.json"

bearer_token = "AAAAAAAAAAAAAAAAAAAAAJzFbwEAAAAAiw1m4S%2FgkghMs%2BWfiVo6OswWgYE%3DGRQPPybZpQuJ1g2lFvxyH9yogZRQatwb6gT8X8ShhyYdjmqXGu"

In [87]:
total_id_list = []
with open(DATA_PATH, "r") as f:
    for line in f:
        line = line.strip() # remove next line
        line = line.split(',') # split into list
        total_id_list.append(line)
total_id_list = [item for sublist in total_id_list for item in sublist] # Flat into a single list
total_id_list = list(set(total_id_list))

In [88]:
len(total_id_list)

28633

## Twitter Part

In [163]:
def Extend_Json(new_dict, File_PATH):
    """Extend a huge json file without loading it to the memory"""
    # if file exists:
    if os.path.exists(File_PATH):
        if new_dict: # if dict has value
            # Source: https://stackoverflow.com/questions/18087397/append-list-of-python-dictionaries-to-a-file-without-loading-it/31224105#31224105?newreg=6b7713ed96df42959bb9443daf7bb8ec
            # two methods
            # with open (File_PATH, mode="r+") as file:
            #     file.seek(0,2)  # 0 byte (0) from the end of file (2)
            #     position = file.tell() -1
            #     file.seek(position)
            #     file.write( ",{}]".format(json.dumps(new_dict)) )
            with open (File_PATH, mode="r+") as file:
                file.seek(os.stat(File_PATH).st_size -1)
                file.write( ",{}]".format(json.dumps(new_dict, indent=2)) )
    else:
        # else, create a new json file
        with open (File_PATH, mode="w") as file:
            json.dump(new_dict, file, indent=2)

In [194]:
def get_twitter(bearer_token, list_ids, JSON_PATH, tweet_fields=["created_at", "author_id", "entities", "withheld", "public_metrics", "lang", "reply_settings", "source"]):
    """Automatically get a list of tweeters, the function auto divide ids into chunks of size 800 to avoid excessive use of API,
        auto create a Json file in the designated path."""
    CHUNK_SIZE = 99
    ## Auto divide into sub-tasks
    list_ids = list(set(list_ids))
    if len(list_ids) > CHUNK_SIZE:
        chunks_of_list = np.array_split(list_ids, int(len(list_ids)/CHUNK_SIZE))
    else:
        chunks_of_list = list_ids
    
    loop_count = 1
    for sub_list in chunks_of_list:
        ## Main loop to get twitter by chunks
    
        output_list = []   # output list of dictionary of tweets
        author_id_list = []  # store author ids to produce author database

        client = tweepy.Client(bearer_token)

        # Get Tweets

        # This endpoint/method returns a variety of information about the Tweet(s)
        # specified by the requested ID or list of IDs

        tweet_ids = list(sub_list)

        # By default, only the ID and text fields of each Tweet will be returned
        # Additional fields are retrieved using the tweet_fields parameter, selected by hands to make sure we capture useful informations
        response = client.get_tweets(tweet_ids, tweet_fields=tweet_fields)

        for tweet in response.data:
            # set withheld to true or false based on return
            if tweet.withheld:
                withheld = True
            else:
                withheld = False
            tweet_dict = {
                "id":              tweet.id,
                "created_at":      str(tweet.created_at),
                "author_id":       tweet.author_id, 
                "lang":            tweet.lang,
                "reply_settings":  tweet.reply_settings,
                "source":          tweet.source,
                "withheld":        withheld,
                "public_metrics":  tweet.public_metrics,
                "entities":        tweet.entities,
            }
            output_list.append(tweet_dict)
            author_id_list.append(tweet.author_id) # record author informations as well
        
        # Save to json file
        Extend_Json(output_list, JSON_PATH)
        # Sleep 15 minutes to avoid API restrictions after processing 900 tweets
        #if loop_count == 8: 
            #time.sleep(15*60)
            #loop_count = 0
        loop_count += 1

In [ ]:
get_twitter(bearer_token, total_id_list, JSON_PATH)

## Author Data Part

In [72]:
tweet.source

'Twitter Web App'